In [1]:
#просмотрим все текущие соревнования
!kaggle competitions list

ref                                                  deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                         2030-07-01 23:59:00  Getting Started     Prizes         66           False  
gan-getting-started                                  2030-07-01 23:59:00  Getting Started     Prizes        137           False  
tpu-getting-started                                  2030-06-03 23:59:00  Getting Started  Knowledge        187           False  
digit-recognizer                                     2030-01-01 00:00:00  Getting Started  Knowledge       1558           False  
titanic                                              2030-01-01 00:00:00  Getting Started  Knowledge      49630            True  
house-prices-advanced-regression-techniques          2030-01-01 00:00:00  Getting Started 

In [2]:
#смотрим файлы выбранного соревнования
!kaggle competitions files real-estate-price-prediction-moscow                                            

name                    size  creationDate         
---------------------  -----  -------------------  
test.csv               475KB  2020-09-16 09:01:30  
sample_submission.csv   70KB  2020-09-16 09:01:30  
train.csv                1MB  2020-09-16 09:01:30  


In [3]:
#выкачиваем файлы выбранного соревнования
!kaggle competitions download real-estate-price-prediction-moscow

real-estate-price-prediction-moscow.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
TRAIN_PATH = './Data/train.csv'
TEST_PATH = './Data/test.csv'

In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
df_train = pd.read_csv(TRAIN_PATH) 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

### План анализа данных:

1.   Загрузить данные для обучения
2.   Обработать данные перед обучением модели
- выбросы очистить
- пропуски убрать
3.   Обучить модель на обучающей выборке
4.   Загрузить и предобработать данные для тестирования
5.   Провалидировать модель на тестовой выборке




## 1. Загрузить данные для обучения

**Шаг 1.1. Загружаем библиотеки** 

In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; 
%matplotlib inline 
import seaborn
import sklearn

**Шаг 1.2. Загрузим данные**

In [8]:
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

# 2. Обработаем данные перед обучением модели

**Шаг 2.1. Проверяем данные на наличие пропусков и типов переменных**

In [10]:
df_train = df_train.drop(['Ecology_2', 'Ecology_3','Shops_2'], axis=1)
df_test = df_test.drop(['Ecology_2', 'Ecology_3','Shops_2'], axis=1)
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Social_1       10000 non-null  int64  
 11  Social_2       10000 non-null  int64  
 12  Social_3       10000 non-null  int64  
 13  Healthcare_1   5202 non-null   float64
 14  Helthcare_2    10000 non-null  int64  
 15  Shops_1        10000 non-null  int64  
 16  Price          10000 non-null  float64
dtypes: float64(8), int64(9)
memory usage: 1.3 MB
<class

Анализируем результата выполнения команды:

* 10000 строк (entries)
* 14 столбцов (Data columns)

В данных присутствует 2 типа dtypes:
* int64 - целое число  (5 столбцов)
* float64 - дробное число (3 столбца)

Почистим данные, в которых нет значений (жилую площадь посчитаем как разницу общей и кухни)

In [11]:
training_data = df_train.copy()
training_data.loc[(training_data['Healthcare_1'].isna()) , 'Healthcare_1'] = training_data['Healthcare_1'].median()
training_data.loc[(training_data['LifeSquare'].isna()) , 'LifeSquare'] = training_data['Square'] - training_data['KitchenSquare']
df_train = training_data.copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     10000 non-null  float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Social_1       10000 non-null  int64  
 11  Social_2       10000 non-null  int64  
 12  Social_3       10000 non-null  int64  
 13  Healthcare_1   10000 non-null  float64
 14  Helthcare_2    10000 non-null  int64  
 15  Shops_1        10000 non-null  int64  
 16  Price          10000 non-null  float64
dtypes: float64(8), int64(9)
memory usage: 1.3 MB


In [12]:
testing_data = df_test.copy()
testing_data.loc[(testing_data['Healthcare_1'].isna()) , 'Healthcare_1'] = testing_data['Healthcare_1'].median()
testing_data.loc[(testing_data['LifeSquare'].isna()) , 'LifeSquare'] = testing_data['Square'] - testing_data['KitchenSquare']
df_test = testing_data.copy()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     5000 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Social_1       5000 non-null   int64  
 11  Social_2       5000 non-null   int64  
 12  Social_3       5000 non-null   int64  
 13  Healthcare_1   5000 non-null   float64
 14  Helthcare_2    5000 non-null   int64  
 15  Shops_1        5000 non-null   int64  
dtypes: float64(7), int64(9)
memory usage: 625.1 KB


**Шаг 2.2. Удаляем пропуски**

Как мы уже видели выше, в наших данных есть пропуски (значения NaN). Для удобства работы выкинем такие данные из нашего датасета, применив метод **dropna()** к *training_data*:

**Шаг 2.3. Отделяем текстовые признаки от числовых**

Мы видим, что некоторые характеристики представлены текстовыми значениями. Машине сложно обрабатывать текстовые признаки, поэтому для упрощения работы мы оставим столбцы, содержащие только числовые данные.   

Отметим, что, таким образом мы можем потерять довольно много информации (например, очевидно, что тип привода (признак drive)  влияет на стоимость), однако мы сознательно идем на это упрощение на данном этапе обучения. 

Чтобы получить все числовые характеристики, необхдимо применить метод **_get_numeric_data()** к объекту *training_data*

In [13]:
df_train.head(5)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
0,11809,27,3.0,115.027311,105.027311,10.0,4,10.0,2014,0.075424,11,3097,0,900.0,0,0,305018.871089
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,30,6207,1,1183.0,1,0,177734.553407
2,8215,1,3.0,78.342215,47.671972,10.0,2,17.0,1988,0.025609,33,5261,0,240.0,3,1,282078.720850
3,2352,1,1.0,40.409907,39.409907,1.0,10,22.0,1977,0.007122,1,264,0,900.0,0,1,168106.007630
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,33,8667,2,900.0,0,6,343995.102962


In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     10000 non-null  float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Social_1       10000 non-null  int64  
 11  Social_2       10000 non-null  int64  
 12  Social_3       10000 non-null  int64  
 13  Healthcare_1   10000 non-null  float64
 14  Helthcare_2    10000 non-null  int64  
 15  Shops_1        10000 non-null  int64  
 16  Price          10000 non-null  float64
dtypes: float64(8), int64(9)
memory usage: 1.3 MB


**Шаг 2.4. Работаем с целевой переменной**

*Какая переменная целевая?*

В данном случае по условию задачи мы должны прогнозировать стоимость квартиры, поэтому целевая переменная - это price. Запишем название целевого признака в отдельную переменную с названием ***target_variable_name***. 

Нам нужно выделить в отдельную переменную ***training_values*** столбец из нашей таблицы, который соответствует определенной выше целевой переменной. Для этого мы у таблицы *training_data* в квадратных скобках указываем имя нужного столбца. В нашем случае это имя записано в переменной *target_variable_name*. 

In [15]:
y_train = df_train['Price']
y_train

0       305018.871089
1       177734.553407
2       282078.720850
3       168106.007630
4       343995.102962
            ...      
9995    119367.455796
9996    199715.148807
9997    165953.912580
9998    171842.411855
9999    177685.627486
Name: Price, Length: 10000, dtype: float64

<font color='blue'>Отделим входные переменные от выходной (целевой), чтобы можно было построить модель предсказания целевой переменной по входным.  

Для это нужно у переменной ***df_train*** вызвать метод **drop()**. Результат мы записываем в новую переменную ***training_points***. После выполнения запроса ***training_points*** будет содержать исходную таблицу без целевого столбца. 
 

Обратите внимание, что в данном случае мы передаем два аргумента:
    1. ***target_variable_name*** - название столбца цены, который мы ранее записали в эту переменную и теперь хотим удалить из training_data
    2. axis=1 - означает, что мы удаляем столбец, а в случае axis=0 - означает, что мы удаляем строку

In [16]:
x_train = df_train.drop('Price', axis=1)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     10000 non-null  float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Social_1       10000 non-null  int64  
 11  Social_2       10000 non-null  int64  
 12  Social_3       10000 non-null  int64  
 13  Healthcare_1   10000 non-null  float64
 14  Helthcare_2    10000 non-null  int64  
 15  Shops_1        10000 non-null  int64  
dtypes: float64(7), int64(9)
memory usage: 1.2 MB


##   3. Обучить модель на обучающей выборке

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
...     x_train, y_train, test_size=0.3, random_state=42)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)

(7000, 16)
(3000, 16)
(7000,)
(3000,)
(10000, 16)


In [61]:
print(X_train.info())
print(X_test.info())
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 9069 to 7270
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             7000 non-null   int64  
 1   DistrictId     7000 non-null   int64  
 2   Rooms          7000 non-null   float64
 3   Square         7000 non-null   float64
 4   LifeSquare     7000 non-null   float64
 5   KitchenSquare  7000 non-null   float64
 6   Floor          7000 non-null   int64  
 7   HouseFloor     7000 non-null   float64
 8   HouseYear      7000 non-null   int64  
 9   Ecology_1      7000 non-null   float64
 10  Social_1       7000 non-null   int64  
 11  Social_2       7000 non-null   int64  
 12  Social_3       7000 non-null   int64  
 13  Healthcare_1   7000 non-null   float64
 14  Helthcare_2    7000 non-null   int64  
 15  Shops_1        7000 non-null   int64  
dtypes: float64(7), int64(9)
memory usage: 929.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Int

**Шаг 3.1. Выбираем метод, который будем использовать**

Проще всего начать с простых методов. 
Мы воспользуемся двумя методами для построения моделей и сравним их между собой:

<font color='green'> **Линейная регрессия** <font color='red'> *linear regression*

<font color='green'> **Лес решающих деревьев** <font color='red'> *random forest*

На выбор метода для построения модели влияет набор признаков, размер выборки, интуиция про то, какая связь между входными переменными и целевой. Но часто решение принимается исходя из того, какая модель сработала лучше.

Для корректной работы с методами построения моделей в python требуется загрузить специальную библиотеку
**sklearn**, программную библиотеку на языке python для для машинного обучения и анализа данных.

Мы импортируем два модуля из этой библиотеки:
 * *linear_model* - тут находятся все линейные модели
 * *ensemble* - тут находятся модели на основе ансамблей

In [19]:
import sklearn

In [20]:
from sklearn import linear_model, ensemble

Чтобы создать модель **линейной регресии**, пишем имя модуля 'linear_model', затем точку, затем название модели.

Для этого нужно выполнить следующий код:


In [21]:
#  создадим модель линейной регресси
linreg_model = linear_model.LinearRegression()
linreg_model

LinearRegression()

In [22]:
#  создадим модель случайного леса
random_forest_model = ensemble.RandomForestRegressor()
random_forest_model

RandomForestRegressor()

**Шаг 3.2. Обучим модель**

Теперь, когда мы создали прототипы обеих моделей, можем их обучить с помощью обучающей выборки. 

Для этого вызываем метод **fit()** у каждой модели и передаем ему на вход два аргумента: 
таблицу входных признаков и столбец значений целевой переменной - (training_points, training_values)

In [23]:
linreg_model.fit(X_train, y_train)

LinearRegression()

Делаем тоже самое для модели решающего леса.

In [24]:
random_forest_model.fit(X_train, y_train)

RandomForestRegressor()

* Для двух разных моделей в sklearn методы для обучения модели не отличаются.
* Мы получили две обученные модели. 
* Теперь необходимо провалидировать модели на новых тестовых данных. 

## 4. Загрузить и предобработать данные для тестирования

**Шаг 4.1. Загрузим и проанализируем тестовые данные.**

В функции передаем один атрибут: название файла, в котором находится таблица с данными.

**Шаг 4.2. Отделяем целевую переменную**




Нам нужно выделить в отдельную переменную ***test_values*** столбец из нашей таблицы, который соответствует определенной выше целевой переменной. Для этого мы у таблицы *test_data* в квадратных скобках указываем имя нужного столбца. В нашем случае это имя записано в переменной ***target_variable_name***. 

# 5. Провалидировать модель на тестовой выборке

**Шаг 5.1. Сравнение моделей.**

Теперь мы готовы сравнить качество двух моделей! 😎 :) 

*1. Какая модель лучше?*

Получим прогнозы целевой переменной на тестовых данных для модели линейной регрессии м модели случайного леса. 

Для этого вызовем у каждой модели метод **predict()**, в качестве аргумента передадим *test_points*.

In [47]:
predict_linreg_model = linreg_model.predict(X_test)
len(predict_linreg_model)

3000

In [48]:
predict_random_forest = random_forest_model.predict(X_test)
len(predict_random_forest)

3000

<font color=green>Качество регрессионных моделей оценим двумя способами: 
1. Сравним визуально прогнозы с настоящими ценами (тестовые с предсказанием)
2. Сравним метрики качества

<font color=green>Визуализируем прогноз линейной модели и настоящие значения из тестовой выборки.
Можно не обращать особого внимания на то, что за код написан.

Визуализируем прогноз модели случайного леса и настоящие значения из тестовой выборки

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Подсчитаем ошибки для линейной модели.

Для этого вызовем методы **mean_absolute_error()** и **mean_squared_error()**. На вход им передается столбец настоящих значений *test_values* и столбец значений, предсказанных моделью линейной регрессии *test_predictions_linear*.

In [53]:
mean_absolute_error_linear_model = mean_absolute_error(y_test, predict_linreg_model) 
mean_squared_error_linear_model = mean_squared_error(y_test, predict_linreg_model)

для леса аналогичные ошибки

In [49]:
mean_absolute_error_random_forest_model = mean_absolute_error(y_test, predict_random_forest)
mean_squared_error_random_forest_model = mean_squared_error(y_test, predict_random_forest)

Теперь напечатаем полученные ошибки. Обычно смотрят на корень из среднеквадратичной ошибки, RMSE. Чтобы извлечь корень нам понадобится библиотека **Numpy**. С помощью неё можно быстро производить вычисления сразу над массивами чисел.

In [54]:
import numpy as np

print("MAE: {0:7.2f}, RMSE: {1:7.2f} для модели линейной регрессии".format(
        mean_absolute_error_linear_model, 
        np.sqrt(mean_squared_error_linear_model)))

print("MAE: {0:7.2f}, RMSE: {1:7.2f} для модели случайного леса".format(
       mean_absolute_error_random_forest_model, 
       mean_squared_error_random_forest_model**(1/2)))

MAE: 48508.09, RMSE: 128215.25 для модели линейной регрессии
MAE: 29772.53, RMSE: 50098.08 для модели случайного леса


## Модель случайного леса круче. Загрузим в нее тестовую выборку (из задания)

In [57]:
# Она уже у нас почищена и подготовлена в датафрейм
df_test.head(3)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
0,4567,44,1.0,36.847630,19.094182,5.0,5,9.0,1970,0.036122,24,4378,0,1036.0,1,1
1,5925,62,1.0,42.493907,42.568133,10.0,7,17.0,2017,0.072158,2,629,1,900.0,0,0
2,960,27,2.0,59.463678,50.463678,9.0,19,19.0,1977,0.211401,9,1892,0,900.0,0,1


In [59]:
predict_RF = random_forest_model.predict(df_test)
len(predict_RF)

5000

In [61]:
# из массива результатов предсказания модели сделаем ДатаФрейм
answer_df = pd.DataFrame(predict_RF)
round(answer_df) # - количество великов

,0
0,160368.0
1,119404.0
2,139251.0
3,130800.0
4,200091.0
...,...
4995,184822.0
4996,390522.0
4997,146235.0
4998,185152.0


In [65]:
question_path = './data/sample_submission.csv'
question_df = pd.read_csv(question_path)
question_df

,Id,Price
0,4567,200000.0
1,5925,200000.0
2,960,200000.0
3,3848,200000.0
4,746,200000.0
...,...,...
4995,10379,200000.0
4996,16138,200000.0
4997,3912,200000.0
4998,5722,200000.0


In [86]:
df = pd.concat([question_df,answer_df], axis=1)
df.pop('Price')
df.rename(columns={0: 'Predict_Price'}, inplace=True)
df

,Id,Predict_Price
0,4567,160367.779730
1,5925,119404.085014
2,960,139250.781126
3,3848,130799.783259
4,746,200090.833136
...,...,...
4995,10379,184822.459524
4996,16138,390521.638181
4997,3912,146234.952068
4998,5722,185152.174074


In [87]:
df.to_csv('result_price_predict.csv')